In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime, date
from tqdm import tqdm_notebook as tqdm
import os


from multiprocessing import Pool
from functools import partial
import ALL_STOCK_preprocess_function as func

In [2]:
file_path = 'D:\\OneDrive - SinoPac\\Cmoney資料表\\基本面\\下市櫃公司基本資料\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\下市櫃公司基本資料\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_off_stock, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
off = pd.concat(df_list, axis=0)
off.to_csv('D:\\庫存健診開發\\data\\processed\\Offdate.csv', index=False)

In [3]:
file_path = 'D:\\OneDrive - SinoPac\\Cmoney資料表\\基本面\\上市櫃公司基本資料\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\上市櫃公司基本資料\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_fundamental, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
fundamental = pd.concat(df_list, axis=0)
fundamental.to_csv('D:\\庫存健診開發\\data\\processed\\Ondate.csv', index=False)

In [2]:
file_path = 'D:\\SinoPac\\彭偉綸-數位金融部-證券 - Cmoney資料表\\技術面\\週收盤表排行\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\價量資料\\'
df_list, index_list, industry_list = [], [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.data_preprocess, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
            df_list.append(x[0])
            index_list.append(x[1])
            industry_list.append(x[2])
                
data, index, industry_index = pd.concat(df_list, axis=0), pd.concat(index_list, axis=0), pd.concat(industry_list, axis=0)
data['ts'], index['ts'], industry_index['ts'] = pd.to_datetime(data['ts']), pd.to_datetime(index['ts']), pd.to_datetime(industry_index['ts'])
index.to_csv('D:\\庫存健診開發\\data\\processed\\index_weekly.csv', index=False)
industry_index.to_csv('D:\\庫存健診開發\\data\\processed\\industry_index_weekly.csv', index=False)
data.to_csv('D:\\庫存健診開發\\data\\processed\\TWSE_ALL_weekly.csv', index=False)

In [3]:
fundamental = pd.read_csv('D:\\庫存健診開發\\data\\processed\\Ondate.csv', converters={'StockNo': str})
fundamental['On_Date'] = pd.to_datetime(fundamental['On_Date'])
off = pd.read_csv('D:\\庫存健診開發\\data\\processed\\Offdate.csv', converters={'StockNo': str})

In [4]:
df = func.filter_stocks(data, fundamental, off)
df

D:\庫存健診開發\code\Preprocess\ALL_STOCK_preprocess_function.py:164: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.ts.dt.date > date(2007,7,1)][df.ts.dt.date < date(2019,9,24)]


,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,total,capital,VWAP,On_Date
171,2007-07-06,1101,台泥,38.20,38.65,36.75,38.55,1.18,4.99,71735,2687679,1236.5,37.466774,1962-02-09
173,2007-07-13,1101,台泥,39.00,39.30,37.30,38.30,-0.65,5.19,69418,2660721,1228.4,38.328978,1962-02-09
174,2007-07-20,1101,台泥,39.00,42.30,38.50,40.35,5.35,9.92,107710,4383515,1294.2,40.697382,1962-02-09
175,2007-07-27,1101,台泥,40.35,44.05,39.85,39.85,-1.24,10.41,79990,3365311,1278.1,42.071646,1962-02-09
176,2007-08-10,1101,台泥,37.00,42.55,37.00,39.45,3.00,14.49,118980,4877686,1281.6,40.995848,1962-02-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628006,2019-09-20,4572,駐龍,156.00,157.50,147.00,151.50,-3.19,6.71,1682,256066,54.2,152.239001,2019-09-10
628009,2019-09-12,4576,大銀微系統,86.40,88.40,83.30,87.00,0.46,5.89,3035,260666,102.6,85.886656,2019-09-04
628013,2019-09-20,4576,大銀微系統,86.50,87.80,84.90,86.20,-0.92,3.33,1257,108359,101.6,86.204455,2019-09-04
628016,2019-09-06,4576,大銀微系統,83.00,93.00,80.00,86.60,NaN,NaN,9838,859028,102.1,87.317341,2019-09-04


In [10]:
file_path = 'D:\\SinoPac\\彭偉綸-數位金融部-證券 - Cmoney資料表\\基本面\\減資狀況\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\減資資料\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_eliminate, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
eliminate = pd.concat(df_list, axis=0)
eliminate = eliminate.sort_values(by='year')
eliminate.to_csv('D:\\庫存健診開發\\data\\processed\\eliminate.csv', index=False)

In [4]:
file_path = 'D:\\SinoPac\\彭偉綸-數位金融部-證券 - Cmoney資料表\\籌碼面\\日法人持股估計\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\三大法人\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_inventory, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
inventory = pd.concat(df_list, axis=0)
inventory = inventory.sort_values(by='ts')
inventory['ts'] = pd.to_datetime(inventory['ts'])
inventory.to_csv('D:\\庫存健診開發\\data\\processed\\inventory.csv', index=False)